In [4]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import xarray as xr
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


# Load SST anomaly data from the text file
sst_data = pd.read_csv('./nino34.long.anom.data.txt', delim_whitespace=True, header=None)
sst_data.columns = ['Year'] + [f'Month_{i}' for i in range(1, 13)]
sst_data.set_index('Year', inplace=True)

# Load SST data from the NetCDF file using xarray
try:
    ds = xr.open_dataset('./sst.mon.mean.trefadj.anom.1880to2018.nc')
    sst_anomalies = ds['sst'].values  # Assuming 'sst' is the variable name
    print(sst_anomalies.shape)
except OSError as e:
    print(f"Error reading NetCDF file: {e}")

# Preprocess and label the data
def label_event(sst_anomaly):
    if sst_anomaly > 0.5:
        return 2  # El Niño
    elif sst_anomaly < -0.5:
        return 0  # La Niña
    else:
        return 1  # Neutral

# Create labels for each month
labels = np.vectorize(label_event)(sst_data.values)

# Reshape the data for CNN input
X = sst_data.values.reshape(-1, 12, 1)  # One year as one sample with 12 months as features
y = labels.reshape(-1, 12)

# Custom dataset class
class SSTDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset and dataloader
dataset = SSTDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Error reading NetCDF file: [Errno -101] NetCDF: HDF error: b'/Users/ludong/PROGRAM/112-2/Statistics/hw9/sst.mon.mean.trefadj.anom.1880to2018.nc'


In [ ]:


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 2, 128)  # 2 comes from the reduced size after conv and pooling layers
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 3)  # 3 classes: El Niño, La Niña, neutral

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 2)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Create model, define loss function and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0
    # Create a progress bar for the epoch
    with tqdm(total=len(dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels[:, 0])  # labels[:, 0] to match the output shape
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update running loss and progress bar
            running_loss += loss.item()
            pbar.set_postfix({'loss': running_loss / (pbar.n + 1)})
            pbar.update(1)
            
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print('Finished Training')


In [ ]:
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels[:, 0]).sum().item()

print(f'Accuracy of the model: {100 * correct / total:.2f}%')
